In [1]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq

from IPython.display import Image, display

import os
from dotenv import load_dotenv

from langchain_tavily import TavilySearch

In [2]:
# Load .env file
load_dotenv()

# Initialize State 
class State(TypedDict):
    messages: Annotated[list, add_messages]

### LLMs

In [3]:
# LLMs
llm_google = ChatGoogleGenerativeAI(model='gemini-2.5-flash-lite')
llm_groq = ChatGroq(model='llama3-8b-8192')

llm = llm_google # Set default llm

### TOOLS

In [4]:
search_tool = TavilySearch(max_results=2)
search_tool.invoke("Agentic AI for beginners")

{'query': 'Agentic AI for beginners',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://learn.microsoft.com/en-us/shows/ai-agents-for-beginners/',
   'title': 'AI Agents for Beginners | Microsoft Learn',
   'content': 'Read in English # AI Agents for Beginners * English In this 10-lesson course we take you from concept to code while covering the fundamentals of building AI agents. * AI Agents for Beginners * AI Agents for Beginners What are AI agents? (Part 1 of 10) | AI Agents for Beginners + Beginner + Beginner + Beginner (Part 4 of 10) | AI Agents for Beginners (Part 5 of 10) | AI Agents for Beginners + Beginner + Beginner (Part 7 of 10) | AI Agents for Beginners + Beginner How can AI agents improve? (Part 9 of 10) | AI Agents for Beginners + Beginner How to deploy AI agents into production (Part 10 of 10) | AI Agents for Beginners + Beginner',
   'score': 0.98594,
   'raw_content': None},
  {'url': 'https://www.freecodecamp.org/news/the-age

In [5]:
# Tool creation using a custom function
def calculate_bill_split(total_bill: float, num_people: int, tip_percentage: int) -> float:
    """
    Calculates the split amount for each person in the bill including the tip.
    
    Args:
        total_bill (float): The total bill amount.
        num_people (int): The number of people sharing the bill.
        tip_percentage (int): The tip percentage to be added to the bill.
    
    Returns:
        float: The amount each person needs to pay.
        
    """
    if num_people == 0:
        return 0.0
    total_with_tip = total_bill + (total_bill * tip_percentage / 100)
    return total_with_tip / num_people  

In [6]:
tools = [search_tool, calculate_bill_split]

In [7]:
# binding llm with tools
llm_with_tools = llm.bind_tools(tools)

### GRAPH

Changes: 
-The edge from tools to END will instead go back from tools to the tool_calling_llm


In [8]:
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}

In [9]:
'''
BUUILDING THE GRAPH
'''
graph_builder = StateGraph(State)

# Nodes
graph_builder.add_node("tool_call_llm", tool_calling_llm)
graph_builder.add_node("tools", ToolNode(tools))

# Edges
graph_builder.add_edge(START, "tool_call_llm")
graph_builder.add_conditional_edges(
    "tool_call_llm",
    tools_condition
    
)
graph_builder.add_edge("tools", "tool_call_llm") # Loop back to tool_call_llm after using a tool

# Compile
graph = graph_builder.compile()

In [18]:
#visualize Graph
def display_graph():
    try:
        display(Image(graph.get_graph().draw_mermaid_png()))
    except Exception as e:
        print(f"Error displaying graph: {e}")

display_graph()


Error displaying graph: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`


### Execute

In [11]:
response = graph.invoke({"messages": "How much do we have to split for our 53.98 USD bill if there are 4 of us and we want to tip 15%? Also, can you give me recent innovations in the field of AI?"})
print(response['messages'][-1].content)

Each person has to pay $15.52.

Here are some recent innovations in AI:
* Walmart is using AI to visually identify projects without packaging, which reduces the expense of training.
* InformationWeek is covering recent news and expert commentary on AI innovations.
